# Създайте модел за класификация: Вкусни азиатски и индийски ястия


## Класификатори за кухня 2

В този втори урок за класификация ще разгледаме `повече начини` за класифициране на категорийни данни. Ще научим също за последствията от избора на един класификатор пред друг.

### [**Тест преди лекцията**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/23/)

### **Предварителни знания**

Предполага се, че сте завършили предишните уроци, тъй като ще използваме някои концепции, които научихме преди.

За този урок ще са ни необходими следните пакети:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) е [колекция от R пакети](https://www.tidyverse.org/packages), създадена да направи науката за данни по-бърза, лесна и забавна!

-   `tidymodels`: [tidymodels](https://www.tidymodels.org/) е рамка, представляваща [колекция от пакети](https://www.tidymodels.org/packages/) за моделиране и машинно обучение.

-   `themis`: [themis пакетът](https://themis.tidymodels.org/) предоставя допълнителни стъпки за обработка на небалансирани данни.

Можете да ги инсталирате със следната команда:

`install.packages(c("tidyverse", "tidymodels", "kernlab", "themis", "ranger", "xgboost", "kknn"))`

Алтернативно, скриптът по-долу проверява дали имате необходимите пакети за завършване на този модул и ги инсталира, ако липсват.


In [ ]:
suppressWarnings(if (!require("pacman"))install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, themis, kernlab, ranger, xgboost, kknn)

## **1. Карта за класификация**

В нашия [предишен урок](https://github.com/microsoft/ML-For-Beginners/tree/main/4-Classification/2-Classifiers-1) се опитахме да отговорим на въпроса: как да изберем между множество модели? До голяма степен това зависи от характеристиките на данните и типа проблем, който искаме да решим (например класификация или регресия).

По-рано научихме за различните опции, които имате при класифициране на данни, използвайки помощния лист на Microsoft. Машинно обучителната рамка на Python, Scikit-learn, предлага подобен, но по-детайлен помощен лист, който може допълнително да помогне за стесняване на избора на оценители (друг термин за класификатори):

<p >
   <img src="../../images/map.png"
   width="700"/>
   <figcaption></figcaption>


> Съвет: [посетете тази карта онлайн](https://scikit-learn.org/stable/tutorial/machine_learning_map/) и кликнете по пътя, за да прочетете документацията.
>
> [Референтният сайт на Tidymodels](https://www.tidymodels.org/find/parsnip/#models) също предоставя отлична документация за различните видове модели.

### **Планът** 🗺️

Тази карта е много полезна, когато имате ясна представа за вашите данни, тъй като можете да „вървите“ по нейните пътеки към решение:

-   Имаме \>50 проби

-   Искаме да предвидим категория

-   Имаме етикетирани данни

-   Имаме по-малко от 100K проби

-   ✨ Можем да изберем Linear SVC

-   Ако това не работи, тъй като имаме числови данни

    -   Можем да опитаме ✨ KNeighbors Classifier

        -   Ако това не работи, опитайте ✨ SVC и ✨ Ensemble Classifiers

Това е много полезен път за следване. Сега, нека се потопим в него, използвайки [tidymodels](https://www.tidymodels.org/) рамката за моделиране: последователна и гъвкава колекция от R пакети, разработени за насърчаване на добри статистически практики 😊.

## 2. Разделяне на данните и справяне с небалансиран набор от данни.

От предишните ни уроци научихме, че има набор от общи съставки в различните кухни. Също така, имаше доста неравномерно разпределение в броя на кухните.

Ще се справим с това, като:

-   Премахнем най-често срещаните съставки, които създават объркване между различните кухни, използвайки `dplyr::select()`.

-   Използваме `recipe`, който предварително обработва данните, за да ги подготви за моделиране, като приложим алгоритъм за `over-sampling`.

Вече разгледахме горното в предишния урок, така че това трябва да бъде лесно 🥳!


In [ ]:
# Load the core Tidyverse and Tidymodels packages
library(tidyverse)
library(tidymodels)

# Load the original cuisines data
df <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/4-Classification/data/cuisines.csv")

# Drop id column, rice, garlic and ginger from our original data set
df_select <- df %>% 
  select(-c(1, rice, garlic, ginger)) %>%
  # Encode cuisine column as categorical
  mutate(cuisine = factor(cuisine))


# Create data split specification
set.seed(2056)
cuisines_split <- initial_split(data = df_select,
                                strata = cuisine,
                                prop = 0.7)

# Extract the data in each split
cuisines_train <- training(cuisines_split)
cuisines_test <- testing(cuisines_split)

# Display distribution of cuisines in the training set
cuisines_train %>% 
  count(cuisine) %>% 
  arrange(desc(n))

### Работа с небалансирани данни

Небалансираните данни често оказват негативно влияние върху представянето на модела. Много модели работят най-добре, когато броят на наблюденията е равен, и следователно имат затруднения с небалансирани данни.

Има основно два начина за справяне с небалансирани набори от данни:

-   добавяне на наблюдения към малцинствения клас: `Over-sampling`, например чрез използване на алгоритъма SMOTE, който синтетично генерира нови примери за малцинствения клас, използвайки най-близките съседи на тези случаи.

-   премахване на наблюдения от мнозинствения клас: `Under-sampling`

В предишния ни урок демонстрирахме как да се справим с небалансирани набори от данни, използвайки `recipe`. Рецептата може да се разглежда като план, който описва какви стъпки трябва да се приложат към набора от данни, за да бъде готов за анализ. В нашия случай искаме да постигнем равномерно разпределение в броя на нашите кухни за нашия `training set`. Нека започнем.


In [ ]:
# Load themis package for dealing with imbalanced data
library(themis)

# Create a recipe for preprocessing training data
cuisines_recipe <- recipe(cuisine ~ ., data = cuisines_train) %>%
  step_smote(cuisine) 

# Print recipe
cuisines_recipe

Сега сме готови да обучаваме модели 👩‍💻👨‍💻!

## 3. Отвъд моделите за мултиномиална регресия

В предишния урок разгледахме модели за мултиномиална регресия. Нека изследваме някои по-гъвкави модели за класификация.

### Поддържащи векторни машини (Support Vector Machines)

В контекста на класификацията, `Поддържащите векторни машини` са техника за машинно обучение, която се опитва да намери *хиперплоскост*, която "най-добре" разделя класовете. Нека разгледаме един прост пример:

<p >
   <img src="../../images/svm.png"
   width="300"/>
   <figcaption>https://commons.wikimedia.org/w/index.php?curid=22877598</figcaption>


H1~ не разделя класовете. H2~ ги разделя, но само с малък отстъп. H3~ ги разделя с максимален отстъп.

#### Линеен класификатор с опорни вектори

Клъстеризацията с опорни вектори (SVC) е част от семейството на машинно обучение с опорни вектори. При SVC хиперплоскостта се избира така, че да разделя `повечето` от тренировъчните наблюдения правилно, но `може да класифицира погрешно` някои наблюдения. Като се позволи на някои точки да бъдат от грешната страна, SVM става по-устойчив към аномалии и следователно по-добре обобщава нови данни. Параметърът, който регулира това нарушение, се нарича `cost` и има стойност по подразбиране 1 (вижте `help("svm_poly")`).

Нека създадем линеен SVC, като зададем `degree = 1` в полиномиален SVM модел.


In [ ]:
# Make a linear SVC specification
svc_linear_spec <- svm_poly(degree = 1) %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svc_linear_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svc_linear_spec)

# Print out workflow
svc_linear_wf

Сега, след като сме обхванали стъпките за предварителна обработка и спецификацията на модела в *workflow*, можем да продължим с обучението на линейния SVC и да оценим резултатите по време на процеса. За метрики на производителността, нека създадем набор от метрики, който ще оцени: `accuracy`, `sensitivity`, `Positive Predicted Value` и `F Measure`.

> `augment()` ще добави колона(и) за предсказания към дадените данни.


In [ ]:
# Train a linear SVC model
svc_linear_fit <- svc_linear_wf %>% 
  fit(data = cuisines_train)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)


# Make predictions and Evaluate model performance
svc_linear_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

#### Поддържаща векторна машина

Поддържащата векторна машина (SVM) е разширение на класификатора с поддържащи вектори, което позволява използването на нелинейна граница между класовете. По същество, SVM използва *трик с ядрото*, за да разшири пространството на характеристиките и да се адаптира към нелинейни връзки между класовете. Една популярна и изключително гъвкава ядрена функция, използвана от SVM, е *функцията с радиална база*. Нека видим как ще се представи върху нашите данни.


In [ ]:
set.seed(2056)

# Make an RBF SVM specification
svm_rbf_spec <- svm_rbf() %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svm_rbf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svm_rbf_spec)


# Train an RBF model
svm_rbf_fit <- svm_rbf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
svm_rbf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Много по-добре 🤩!

> ✅ Моля, вижте:
>
> -   [*Support Vector Machines*](https://bradleyboehmke.github.io/HOML/svm.html), Hands-on Machine Learning with R
>
> -   [*Support Vector Machines*](https://www.statlearning.com/), An Introduction to Statistical Learning with Applications in R
>
> за допълнително четене.

### Класификатори на най-близки съседи

*K*-най-близки съседи (KNN) е алгоритъм, при който всяко наблюдение се предсказва въз основа на неговата *сходство* с други наблюдения.

Нека приложим такъв към нашите данни.


In [ ]:
# Make a KNN specification
knn_spec <- nearest_neighbor() %>% 
  set_engine("kknn") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
knn_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(knn_spec)

# Train a boosted tree model
knn_wf_fit <- knn_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
knn_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Изглежда, че този модел не се представя толкова добре. Вероятно промяната на аргументите на модела (вижте `help("nearest_neighbor")`) ще подобри производителността на модела. Не забравяйте да го изпробвате.

> ✅ Моля, вижте:
>
> -   [Hands-on Machine Learning with R](https://bradleyboehmke.github.io/HOML/)
>
> -   [An Introduction to Statistical Learning with Applications in R](https://www.statlearning.com/)
>
> за да научите повече за класификаторите *K*-най-близки съседи.

### Ансамблови класификатори

Ансамбловите алгоритми работят, като комбинират множество базови оценители, за да създадат оптимален модел, като използват:

`bagging`: прилагане на *усредняваща функция* към колекция от базови модели

`boosting`: изграждане на последователност от модели, които се надграждат един върху друг, за да подобрят предсказателната производителност.

Нека започнем, като изпробваме модел Random Forest, който изгражда голяма колекция от дървета за вземане на решения и след това прилага усредняваща функция за създаване на по-добър цялостен модел.


In [ ]:
# Make a random forest specification
rf_spec <- rand_forest() %>% 
  set_engine("ranger") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
rf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(rf_spec)

# Train a random forest model
rf_wf_fit <- rf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
rf_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Браво 👏!

Нека също така да експериментираме с модел на Усилено Дърво.

Усиленото Дърво определя ансамблов метод, който създава серия от последователни дървета за вземане на решения, като всяко дърво зависи от резултатите на предишните дървета в опит да намали грешката постепенно. То се фокусира върху теглата на неправилно класифицираните елементи и коригира модела за следващия класификатор, за да поправи грешките.

Има различни начини за настройка на този модел (вижте `help("boost_tree")`). В този пример ще настроим Усилени дървета чрез енджина `xgboost`.


In [ ]:
# Make a boosted tree specification
boost_spec <- boost_tree(trees = 200) %>% 
  set_engine("xgboost") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
boost_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(boost_spec)

# Train a boosted tree model
boost_wf_fit <- boost_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
boost_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

> ✅ Моля, вижте:

> -   [Machine Learning for Social Scientists](https://cimentadaj.github.io/ml_socsci/tree-based-methods.html#random-forests)
>
> -   [Hands-on Machine Learning with R](https://bradleyboehmke.github.io/HOML/)
>
> -   [An Introduction to Statistical Learning with Applications in R](https://www.statlearning.com/)
>
> -   <https://algotech.netlify.app/blog/xgboost/> - Разглежда модела AdaBoost, който е добра алтернатива на xgboost.
>
> за да научите повече за ансамбловите класификатори.

## 4. Допълнително - сравнение на множество модели

В тази лаборатория сме създали доста модели 🙌. Може да стане досадно или обременително да създаваме множество работни потоци от различни набори от предварителни обработки и/или спецификации на модели и след това да изчисляваме метриките за производителност една по една.

Нека видим дали можем да решим този проблем, като създадем функция, която прилага списък от работни потоци върху обучаващия набор и след това връща метриките за производителност, базирани на тестовия набор. Ще използваме `map()` и `map_dfr()` от пакета [purrr](https://purrr.tidyverse.org/), за да приложим функции към всеки елемент от списъка.

> [`map()`](https://purrr.tidyverse.org/reference/map.html) функциите ви позволяват да замените много цикли for с код, който е по-кратък и по-лесен за четене. Най-доброто място за учене за [`map()`](https://purrr.tidyverse.org/reference/map.html) функциите е [главата за итерации](http://r4ds.had.co.nz/iteration.html) в R for Data Science.


In [ ]:
set.seed(2056)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)

# Define a function that returns performance metrics
compare_models <- function(workflow_list, train_set, test_set){
  
  suppressWarnings(
    # Fit each model to the train_set
    map(workflow_list, fit, data = train_set) %>% 
    # Make predictions on the test set
      map_dfr(augment, new_data = test_set, .id = "model") %>%
    # Select desired columns
      select(model, cuisine, .pred_class) %>% 
    # Evaluate model performance
      group_by(model) %>% 
      eval_metrics(truth = cuisine, estimate = .pred_class) %>% 
      ungroup()
  )
  
} # End of function

In [ ]:
# Make a list of workflows
workflow_list <- list(
  "svc" = svc_linear_wf,
  "svm" = svm_rbf_wf,
  "knn" = knn_wf,
  "random_forest" = rf_wf,
  "xgboost" = boost_wf)

# Call the function
set.seed(2056)
perf_metrics <- compare_models(workflow_list = workflow_list, train_set = cuisines_train, test_set = cuisines_test)

# Print out performance metrics
perf_metrics %>% 
  group_by(.metric) %>% 
  arrange(desc(.estimate)) %>% 
  slice_head(n=7)

# Compare accuracy
perf_metrics %>% 
  filter(.metric == "accuracy") %>% 
  arrange(desc(.estimate))


Пакетът [**workflowset**](https://workflowsets.tidymodels.org/) позволява на потребителите да създават и лесно да прилагат голям брой модели, но е основно предназначен за работа с техники за повторно извадково тестване като `кръстосана проверка`, подход, който все още не сме разгледали.

## **🚀Предизвикателство**

Всяка от тези техники има голям брой параметри, които можете да настройвате, например `cost` в SVMs, `neighbors` в KNN, `mtry` (Случайно избрани предиктори) в Random Forest.

Проучете стандартните параметри на всяка от тях и помислете какво би означавало настройването на тези параметри за качеството на модела.

За да научите повече за конкретен модел и неговите параметри, използвайте: `help("model")`, например `help("rand_forest")`.

> На практика обикновено *оценяваме* *най-добрите стойности* за тези параметри, като обучаваме много модели върху `симулиран набор от данни` и измерваме колко добре се представят всички тези модели. Този процес се нарича **настройване**.

### [**Тест след лекцията**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/24/)

### **Преглед и самостоятелно обучение**

В тези уроци има много терминология, затова отделете малко време, за да прегледате [този списък](https://docs.microsoft.com/dotnet/machine-learning/resources/glossary?WT.mc_id=academic-77952-leestott) с полезни термини!

#### БЛАГОДАРНОСТИ КЪМ:

[`Allison Horst`](https://twitter.com/allison_horst/) за създаването на невероятните илюстрации, които правят R по-приветлив и ангажиращ. Намерете повече илюстрации в нейния [галерия](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).

[Cassie Breviu](https://www.twitter.com/cassieview) и [Jen Looper](https://www.twitter.com/jenlooper) за създаването на оригиналната версия на този модул на Python ♥️

Приятно учене,

[Eric](https://twitter.com/ericntay), Златен студентски посланик на Microsoft Learn.

<p >
   <img src="../../images/r_learners_sm.jpeg"
   width="569"/>
   <figcaption>Илюстрация от @allison_horst</figcaption>



---

**Отказ от отговорност**:  
Този документ е преведен с помощта на AI услуга за превод [Co-op Translator](https://github.com/Azure/co-op-translator). Въпреки че се стремим към точност, моля, имайте предвид, че автоматичните преводи може да съдържат грешки или неточности. Оригиналният документ на неговия изходен език трябва да се счита за авторитетен източник. За критична информация се препоръчва професионален превод от човек. Ние не носим отговорност за каквито и да е недоразумения или погрешни интерпретации, произтичащи от използването на този превод.
